This code file was written by Kat Marton in google colaboratory
Last editied by Kat 2/28/24

This file explores the package neurokit2 for feature extraction and plots the participant data. While there is feature extraction at the end, this is just exploratory.
The next file, "feature_extraction_windows" uses some of the same code as here, but has less plotting. This file is kept mostly for the graphs and visualizations.

In [1]:
!pip install neurokit2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00


In [2]:
import neurokit2 as nk
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from scipy.signal import find_peaks

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
%cd "/content/drive/MyDrive/coding/data"

Mounted at /content/drive/
/content/drive/MyDrive/coding/data


In [4]:
#function to get peaks for an entire participant's data
def nk2_peaks(subjID):
  f_path = subjID + "_lab_filtered_cleaned.csv"
  df = pd.read_csv(f_path)
  df["Block"] = df["Block"].astype(int)
  df["SubjID"] = subjID

  peaks, info = nk.ppg_peaks(df["padasip"], method="elgendi", sampling_rate=64)
  df["Peaks"] = peaks

  return df

In [10]:
#findpeaks in scipy
#can add optional parameters to the find_peaks function to adjust for minimum height, distance and threshhold

def scipy_peaks(subjID):
  f_path = subjID + "_lab_filtered_cleaned.csv"
  df = pd.read_csv(f_path)
  df["Block"] = df["Block"].astype(int)
  df["SubjID"] = subjID

  peaks = find_peaks(df["padasip"], height=0, distance=26)
  peaks_series = pd.Series(peaks[0])
  df["Peaks"] = 0
  df.loc[peaks_series, "Peaks"] = 1


  return df

In [11]:
#plot with Plotly by participant
def plot_participant(df):

  signal = df["padasip"]
  peak_indices = df.index[df['Peaks']==1].tolist()

  fig = go.Figure(data=go.Scatter(
      y=signal,
      mode='lines'
  ))

  fig.add_trace(go.Scatter(
      x=peak_indices,
      y=[signal[j] for j in peak_indices],
      mode='markers',
      marker=dict(
          size=8,
          color='red',
          symbol='circle'
      ),
     name="Detected Peaks"
  ))

  fig.show()

In [12]:
#plotting with Plotly by block
def plot_block (df, block):

  blockdf = df[df["Block"]==block]
  blockdf.reset_index(inplace=True)
  signal = blockdf["padasip"]
  peak_indices = blockdf.index[blockdf['Peaks']==1].tolist()

  fig = go.Figure(data=go.Scatter(
      y=signal,
      mode='lines'
  ))

  fig.add_trace(go.Scatter(
      x=peak_indices,
      y=[signal[j] for j in peak_indices],
      mode='markers',
      marker=dict(
          size=8,
          color='red',
          symbol='circle'
      ),
     name="Detected Peaks"
  ))

  fig.show()

In [13]:
nk2_cov4 = nk2_peaks("Cov4")
nk2_cov7 = nk2_peaks("Cov7")
nk2_cov8 = nk2_peaks("Cov8")
nk2_cov13 = nk2_peaks("Cov13")
nk2_cov14 = nk2_peaks("Cov14")
nk2_cov20 = nk2_peaks("Cov20")

scipy_cov4 = scipy_peaks("Cov4")
scipy_cov7 = scipy_peaks("Cov7")
scipy_cov8 = scipy_peaks("Cov8")
scipy_cov13 = scipy_peaks("Cov13")
scipy_cov14 = scipy_peaks("Cov14")
scipy_cov20 = scipy_peaks("Cov20")

In [ ]:
#calculating differences between both peak detection methods - how many more peaks does scipy recognize as compared to nk2
sum(nk2_cov4["Peaks"]) #2425
sum(scipy_cov4["Peaks"]) #2746

sum(nk2_cov7["Peaks"]) #2553
sum(scipy_cov7["Peaks"]) #2884

sum(nk2_cov8["Peaks"]) #2497
sum(scipy_cov8["Peaks"]) #2728

sum(nk2_cov13["Peaks"]) #2309
sum(scipy_cov13["Peaks"]) #2997

sum(nk2_cov14["Peaks"]) #2226
sum(scipy_cov14["Peaks"]) #3409

sum(nk2_cov20["Peaks"]) #2215
sum(scipy_cov20["Peaks"]) #3418

3418

Plotting by Participant using NeuroKit2 package peak detection

In [14]:
plot_participant(scipy_cov13)

Output hidden; open in https://colab.research.google.com to view.

In [15]:
plot_participant(scipy_cov20)

Output hidden; open in https://colab.research.google.com to view.

In [16]:
plot_block(scipy_cov13, 1)

In [ ]:
plot_block(scipy_cov20, 3)

In [ ]:
plot_block(nk2_cov20, 6)

In [ ]:
plot_block(scipy_cov20, 6)

**Time Domain Feature Extraction**

In [ ]:
def get_features(x):
  return nk.hrv_time(x, sampling_rate=64)

In [ ]:
#grouping by block
def features_by_block(df):
  by_block = df.groupby(["Block", "SubjID"])['Peaks']
  df = by_block.apply(get_features)
  return df

In [ ]:
cov20 = all_peaks("Cov20")

In [ ]:
#features = pd.concat([features_by_block(cov4),features_by_block(cov7),features_by_block(cov8),features_by_block(cov13),features_by_block(cov14),features_by_block(cov20)])

In [ ]:
#features.to_csv("features_12_11_23.csv")

In [ ]:
features

,,,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
Block,SubjID,,,,,,,,,,,,,,,,,,,,,,
0,Cov4,0,760.165230,121.459636,17.234315,115.734771,8.463008,121.383085,NaN,NaN,169.326120,169.521396,...,1.746709,4.698970,1.267604,1.372904,1.359924,2.000908,1.738912,2.005148,3.466050,1.027606
1,Cov4,0,926.724138,521.951636,220.120836,443.201624,NaN,NaN,NaN,NaN,634.756422,635.756596,...,0.691993,5.169926,0.638709,0.364878,0.850707,0.720083,0.775438,1.980826,2.372142,0.808275
2,Cov4,0,773.585705,178.398423,45.549169,169.630884,NaN,NaN,NaN,NaN,234.653650,234.973090,...,1.279534,4.849125,1.021974,1.237076,1.343554,1.472476,1.342639,1.979109,2.257742,0.924388
3,Cov4,0,865.794940,373.636371,113.077165,340.885638,NaN,NaN,NaN,NaN,536.743268,537.556954,...,1.034193,5.110417,0.735133,0.936798,1.126113,1.086221,0.746382,2.030917,1.986634,0.960985
4,Cov4,0,869.250742,432.048752,134.058571,392.727411,NaN,NaN,NaN,NaN,604.409957,605.311360,...,0.787445,4.961290,0.700113,1.041357,0.943425,0.803170,0.724445,2.010367,2.286152,0.772388
5,Cov4,0,792.432109,137.546695,41.054260,132.978277,NaN,NaN,NaN,NaN,190.620731,190.926527,...,2.009398,4.888360,1.361012,1.310441,1.356378,2.081437,1.772303,2.007647,3.173319,1.032942
6,Cov4,0,779.707404,133.239627,21.156442,133.969256,NaN,NaN,NaN,NaN,166.511114,166.777625,...,1.875390,4.947474,1.404120,1.381314,1.305762,1.964765,1.920549,1.961895,3.718870,1.056640
0,Cov7,0,793.083159,174.280458,28.752190,170.829622,28.593973,171.421420,NaN,NaN,236.710931,236.959335,...,1.925023,5.366542,1.550228,1.110101,1.447136,1.953972,1.812761,1.981446,3.398352,1.098643
1,Cov7,0,817.557566,183.484070,15.865649,182.400332,21.215835,179.861712,NaN,NaN,260.948226,261.288459,...,2.109540,5.398719,1.545670,1.051312,1.378875,2.094731,1.810350,1.990735,2.980117,0.992299
